In [54]:
import pandas as pd
import re
from rapidfuzz import fuzz, process

coaches = pd.read_csv('../data/raw/coaches.csv')
players = pd.read_csv('../data/raw/players.csv')
lastschools = pd.read_csv('../data/raw/lastschools.csv')


In [67]:
print(coaches.head())
print(players.head())
print(schools.head())

                  School              Name                    Title
0  University Of Florida  Kevin O'Sullivan               Head Coach
1  University Of Florida   Chuck Jeroloman     Associate Head Coach
2  University Of Florida      Taylor Black          Assistant Coach
3  University Of Florida        David Kopp           Pitching Coach
4  University Of Florida     Michael Byrne  Student Assistant Coach
                  School           Name  Jersey Position Class Year   Height  \
0  University Of Florida  Justin Nadeau       1   INF/OF        Jr.   6' 0''   
1  University Of Florida       Ty Evans       2       OF        Sr.   6' 2''   
2  University Of Florida     Kyle Jones       3       OF        So.   6' 3''   
3  University Of Florida   Cade Kurland       4      INF        Jr.  5' 11''   
4  University Of Florida      Blake Cyr       5   OF/INF        Jr.  5' 11''   

    Weight                   High School  
0  195 lbs  Bartram Trail (Jacksonville)  
1  215 lbs            Lak

In [68]:
counts = players['Name'].value_counts()
multiple_player_names = counts[counts > 1]
multiple_player_names

Name
Joey McMannis      2
Cameron Johnson    2
Name: count, dtype: int64

In [69]:
high_schools = players['High School'].dropna().unique()
len(high_schools)

444

In [70]:
temp_schools = high_schools[:5]
for school in temp_schools:
    if school == 'N/A':
        continue

    matches = process.extract(
        school,
        temp_schools,
        scorer = fuzz.ratio,
        score_cutoff = 80
    )

    print(matches[0])

('Bartram Trail (Jacksonville)', 100.0, 0)
('Lakeland Christian', 100.0, 1)
('North Oconee (Stetson)', 100.0, 2)
('Berkeley Prep', 100.0, 3)
('Windermere (Miami)', 100.0, 4)


In [79]:
def dedup_high_schools(high_schools, canonical_hs, score_cutoff):
    hs_mappings = {}
    for school in high_schools:
        if school == 'N/A':
            continue

        matches = process.extract(
            school,
            canonical_hs,
            scorer = fuzz.partial_ratio,
            score_cutoff = score_cutoff
        )

        if matches:
            hs_mappings[school] = matches[0][0]
        else:
            hs_mappings[school] = school

    return hs_mappings

In [ ]:
for score_cutoff in [95]:
    hs_mappings = {}
    print(hs_mappings)
    canonical_hs = schools[schools['school type'] == 'high school']['name'].unique()
    high_schools = players['High School'].dropna().unique()
    hs_mappings = dedup_high_schools(high_schools, canonical_hs, score_cutoff=score_cutoff)

    for key, vals in hs_mappings.items():
        if key != vals:
            print(f"{key!r} -> {vals!r}")

    print("\n")

{}
'Lakeland Christian (Virginia Tech)' -> 'Lakeland Christian'
'TNXL Academy (Charleston Southern)' -> 'TNXL Academy'
'Liberty HS' -> 'Liberty'
'John Carroll Catholic HS' -> 'Catholic HS'
'Brandon HS' -> 'Brandon'
'Franklin HS' -> 'Franklin'
'Pearland HS' -> 'Pearl'
'Columbia River' -> 'Columbia'
'Buchanan HS' -> 'Buchanan'
'Sherwood HS' -> 'Sherwood'
'Aquinas HS' -> 'Aquinas'
'Liberty-Wentzville' -> 'Liberty'
'East Ridge High School' -> 'East Ridge'
'Austin Prep' -> 'Austin'
'Wayzata HS' -> 'Wayzata'
'Montgomery Lake Creek' -> 'Lake Creek'
'Trussville' -> 'Hewitt-Trussville'
'TNXL Academy [Fla.]' -> 'TNXL Academy'




: 

In [75]:
hs_mappings = {}

for school in high_schools:
    if school == 'N/A':
        continue

    matches = process.extract(
        school,
        canonical_hs,
        scorer = fuzz.partial_ratio,
        score_cutoff = 70
    )

    if matches:
        hs_mappings[school] = matches[0][0]
    else:
        hs_mappings[school] = school

for key, vals in hs_mappings.items():
    if key != vals:
        print(f"{key!r} -> {vals!r}")

'Lakeland Christian (Virginia Tech)' -> 'Lakeland Christian'
'TNXL Academy (Charleston Southern)' -> 'TNXL Academy'
'Liberty HS' -> 'Liberty'
'John Carroll Catholic HS' -> 'Catholic HS'
'Brandon HS' -> 'Brandon'
'De La Salle HS' -> 'De La Salle'
'Franklin HS' -> 'Franklin'
'Pearland HS' -> 'Pearl'
'Columbia River' -> 'Columbia'
'Santa Margarita Catholic HS' -> 'Catholic HS'
'Buchanan HS' -> 'Buchanan'
'Sherwood HS' -> 'Sherwood'
'Centennial HS' -> 'Centennial'
'Columbia River HS' -> 'Columbia'
'Cardinal Newman HS' -> 'Cardinal Newman'
'Aquinas HS' -> 'Aquinas'
'Liberty-Wentzville' -> 'Liberty'
'East Ridge High School' -> 'East Ridge'
'Austin Prep' -> 'Austin'
'Wayzata HS' -> 'Wayzata'
'Montgomery Lake Creek' -> 'Lake Creek'
'Trussville' -> 'Hewitt-Trussville'
'TNXL Academy [Fla.]' -> 'TNXL Academy'


In [77]:
def dedup_coaches(coaches_df):
    # 1. John Smith at ABC Univ: Head Coach
    # 2. John Smith at ABC Univ: Recruiting Coordinator
    # => 1. John Smith at ABC Univ: Head Coach | Recruiting Coordinator
    coaches_clean = coaches_df.groupby(['Name', 'School'], as_index=False).agg({
        'Title': lambda x: ' | '.join(sorted(set(x)))
    })
    return coaches_clean

print(coaches.head())
coaches_clean = dedup_coaches(coaches)
print(coaches_clean)

                  School              Name                    Title
0  University Of Florida  Kevin O'Sullivan               Head Coach
1  University Of Florida   Chuck Jeroloman     Associate Head Coach
2  University Of Florida      Taylor Black          Assistant Coach
3  University Of Florida        David Kopp           Pitching Coach
4  University Of Florida     Michael Byrne  Student Assistant Coach
                 Name                   School  \
0       Adam  Pallone       Rutgers University   
1         Adam Nelson   University of Maryland   
2       Alec Crawford  University Of Minnesota   
3         Alex Corbin   University of Maryland   
4      Amanda Branson    University Of Alabama   
..                ...                      ...   
150  Tyler Schwalbach   University Of Oklahoma   
151     Vince Rinaldi   University Of Oklahoma   
152          Will Fox     Texas A&M University   
153       Zach Cronin    University Of Florida   
154  Zach Weatherford    University Of Ind

In [14]:
coaches = pd.read_csv('../data/raw/coaches.csv')
players = pd.read_csv('../data/raw/players.csv')
schools = pd.read_csv('../data/raw/schools.csv')

def normalize_school_name(name):
    if pd.isna(name) or str(name).strip().upper() == 'N/A':
        return None

    # Lowercase, remove punctuation, collapse whitespace
    name = str(name).lower()
    name = re.sub(r'[^\w\s]', '', name)
    name = re.sub(r'\s+', ' ', name).strip()
    return name

def dedup_high_schools(players_df, schools_df, score_cutoff=80):
    canonical_hs = schools_df[schools_df['school type'] == 'high school']['name'].unique()
    canonical_hs_norm = [normalize_school_name(n) for n in canonical_hs if normalize_school_name(n)]

    hs_names = players_df['High School'].unique()

    hs_mapping = {}

    for orig in hs_names:
        norm = normalize_school_name(orig)
        if not norm:
            hs_mapping[orig] = orig
            continue
        matches = process.extract(
            norm,
            canonical_hs_norm,
            scorer=fuzz.token_sort_ratio,
            score_cutoff=score_cutoff,
        )
        if matches:
            idx = canonical_hs_norm.index(matches[0][0])
            hs_mapping[orig] = canonical_hs[idx]
            # print(f"Fuzzy match: '{orig}' -> '{canonical_hs[idx]}' (score={matches[0][1]})")
        else:
            hs_mapping[orig] = orig
        
    players_df['High School'] = players_df['High School'].map(hs_mapping).fillna(players_df['High School'])
    schools_df['name'] = schools_df['name'].map(hs_mapping).fillna(schools_df['name'])
    
    schools_df.drop_duplicates(subset=['name'], inplace=True)

    # print(f"Original: {len(hs_names)}")
    # print(f"Canonical: {len(set(hs_mapping.values()))}")
    print(f"After deduplication: {len(hs_names) - len(set(hs_mapping.values()))} duplicates removed.")

    return players_df, schools_df

In [15]:
positions = players['Position'].unique()
positions

array(['INF/OF', 'OF', 'INF', 'OF/INF', 'RHP', 'LHP', 'RHP/1B', 'C',
       'C/OF', 'INF/RHP', 'UTIL', 'OF/RHP', 'UTL', 'OF/LHP', '1B',
       'LHP/OF', 'C/INF', 'LHP/INF', 'OF Is Captain (C)', 'RHP/UTL',
       'RHP/OF', 'RHP/INF', 'OF/C', 'RHP/DH', 'IF', 'UT', 'LHP/1B',
       'C/IF', 'Infielder', 'Infield', 'Right Handed Pitcher',
       'Left Handed Pitcher', 'Utility', 'Outfielder', 'Catcher',
       'Catcher/Infielder'], dtype=object)

In [21]:
def extract_positions(pos_str, mapping):
    parts = re.split(r'[/,|]', pos_str)
    return [mapping.get(p) for p in parts if p]

def standardize_positions(players_df):
    mapping = {
        "OF": "Outfielder",
        "INF": "Infielder",
        "C": "Catcher",
        "RHP": "Right-Handed Pitcher",
        "LHP": "Left-Handed Pitcher",
        "UTIL": "Utility",
        "UTL": "Utility",
        "1B": "First Base",
        "2B": "Second Base",
        "3B": "Third Base",
        "SS": "Shortstop",
        "DH": "Designated Hitter",
    }
    # players_df['Position_List'] = players_df['Position'].apply(lambda x: extract_positions(x, mapping))
    positions = players_df['Position'].apply(lambda x: extract_positions(x, mapping))
    for pos in positions:
        print(pos)
    return players_df

In [22]:
players = standardize_positions(players)

['Infielder', 'Outfielder']
['Outfielder']
['Outfielder']
['Infielder']
['Outfielder', 'Infielder']
['Infielder', 'Outfielder']
['Right-Handed Pitcher']
['Right-Handed Pitcher']
['Infielder']
['Infielder']
['Right-Handed Pitcher']
['Infielder']
['Right-Handed Pitcher']
['Outfielder']
['Left-Handed Pitcher']
['Left-Handed Pitcher']
['Right-Handed Pitcher']
['Right-Handed Pitcher', 'First Base']
['Right-Handed Pitcher']
['Right-Handed Pitcher']
['Outfielder']
['Infielder']
['Left-Handed Pitcher']
['Right-Handed Pitcher']
['Catcher']
['Catcher']
['Outfielder', 'Infielder']
['Left-Handed Pitcher']
['Left-Handed Pitcher']
['Right-Handed Pitcher']
['Catcher', 'Outfielder']
['Right-Handed Pitcher']
['Left-Handed Pitcher']
['Catcher']
['Right-Handed Pitcher']
['Right-Handed Pitcher']
['Right-Handed Pitcher']
['Right-Handed Pitcher']
['Infielder', 'Outfielder']
['Infielder', 'Outfielder']
['Infielder']
['Catcher']
['Outfielder']
['Outfielder', 'Infielder']
['Infielder']
['Right-Handed Pitcher']

In [7]:
def map_team(players_df, coaches_df):
    teams_mapping = {
        'University of Florida': 'Florida Gators',
        'University of Missouri': 'Missouri Tigers',
        'University of Oklahoma': 'Oklahoma Sooners',
        'University of Alabama': 'Alabama Crimson Tide',
        'University of Washington': 'Washington Huskies',
        'University of Oregon': 'Oregon Ducks',
        'University of Indiana': 'Indiana Hoosiers',
        'University of Maryland': 'Maryland Terrapins',
        'Rutgers University': 'Rutgers Scarlet Knights',
        'University of Minnesota': 'Minnesota Golden Gophers',
        'Texas A&M University': 'Texas A&M Aggies',
        'University of Mississippi': 'Ole Miss Rebels',
    }

    players_df['Team'] = players_df['School'].apply(lambda x: teams_mapping.get(x))
    coaches_df['Team'] = coaches_df['School'].apply(lambda x: teams_mapping.get(x))

    return players_df, coaches_df

coaches_df = pd.read_csv('../data/processed/coaches.csv')
coaches_df2 = pd.read_csv('../data/processed/coaches.csv')
coaches_df, coaches_df2 = map_team(coaches_df, coaches_df2)
coaches_df.to_csv('../data/processed/coaches.csv', index=False)

In [7]:
players['B/T'].unique()

array(['L/R', 'R/R', 'L/L', 'B/L', 'B/R', 'S/R', 'R/L', 'S/L', 'R-R',
       'L-L', 'R-L', 'L-R', nan, 'S-R'], dtype=object)

In [8]:
players['B/T'] = players['B/T'].fillna('N/A').astype(str)
players['B/T'] = players['B/T'].apply(lambda x: x.replace('-', '/'))
players['B/T'].unique()

array(['L/R', 'R/R', 'L/L', 'B/L', 'B/R', 'S/R', 'R/L', 'S/L', 'N/A'],
      dtype=object)

In [9]:
players['Class Year'].unique()

array(['Jr.', 'Sr.', 'So.', 'R-Fr.', 'Fr.', 'R-Jr.', 'R-So.', 'Gr.',
       'R-Sr.', 'Gr.+'], dtype=object)

In [10]:
mapping = {
        'Jr.': 'Junior',
        'Sr.': 'Senior',
        'So.': 'Sophomore',
        'R-Fr.': 'Redshirt Freshman',
        'Fr.': 'Freshman',
        'R-Jr.': 'Redshirt Junior',
        'R-So.': 'Redshirt Sophomore',
        'Gr.': 'Graduate',
        'R-Sr.': 'Redshirt Senior',
        'Gr.+': 'Graduate',
}

players['Class Year'] = players['Class Year'].apply(lambda x: mapping.get(x))
players['Class Year'].unique()

array(['Junior', 'Senior', 'Sophomore', 'Redshirt Freshman', 'Freshman',
       'Redshirt Junior', 'Redshirt Sophomore', 'Graduate',
       'Redshirt Senior'], dtype=object)

In [28]:
coaches['Title'].unique()

array(['Head Coach', 'Associate Head Coach', 'Assistant Coach',
       'Pitching Coach', 'Student Assistant Coach', 'Student Coach',
       'Associate Strength and Conditioning Coach (Baseball)',
       'Strength & Conditioning Coach',
       'Head Strength Coach & Video Analytics Coordinator',
       'Associate Coach/Pitching',
       'Assistant Coach/Hitting & Recruiting Coordinator',
       'Assistant Coach/Infield', 'Assistant Coach/Pitching & Catching',
       'Special Assistant to the Head Coach',
       'Head Strength & Conditioning Coach',
       'Assistant Strength & Conditioning Coach',
       'Head Strength and Conditioning Coach for Olympic Sports',
       'Assistant Coach / Recruiting Coordinator',
       'Assistant Coach / Outfield & Base Running Coordinator',
       'Assistant Coach/Recruiting Coordinator',
       'Assistant Coach/Pitching Coach',
       'Bixby Family Head Baseball Coach', 'Volunteer Coach',
       'Assistant Strength and Conditioning Coach--Baseball',
 

In [ ]:
ROLE_KEYWORDS = [
    # Primary roles
    "head coach",
    "associate head coach",
    "assistant coach",
    "pitching coach",
    "hitting coach",
    "recruiting coordinator",
    "strength and conditioning coach",
    "strength & conditioning coach",
    "strength coach",
    "volunteer coach",
    "student assistant coach",
    "student coach",
    "undergraduate assistant coach",
    "special assistant",
]

CANONICAL_MAP = {
    "strength coach": "Strength & Conditioning Coach",
    "strength & conditioning coach": "Strength & Conditioning Coach",
    "strength and conditioning coach": "Strength & Conditioning Coach",
    "assistant coach": "Assistant Coach",
    "associate head coach": "Associate Head Coach",
    "head coach": "Head Coach",
    "volunteer coach": "Volunteer Coach",
    "student assistant coach": "Student Assistant Coach",
    "student coach": "Student Assistant Coach",
    "undergraduate assistant coach": "Student Assistant Coach",
    "special assistant": "Assistant Coach",
}

In [ ]:
def clean_role_list(roles):
    if "Head Coach" in roles:
        roles = [r for r in roles if r not in ["Associate Head Coach", "Assistant Coach"]]
    elif "Associate Head Coach" in roles:
        # Remove Assistant Coach
        roles = [r for r in roles if r != "Assistant Coach"]
    
    if "Student Assistant Coach" in roles and "Assistant Coach" in roles:
        roles.remove("Student Assistant Coach")

    # Remove redundant variants
    if "Pitching Coach" in roles and "Pitching" in roles:
        roles.remove("Pitching")
    elif "Pitching" in roles:
        roles.append("Pitching Coach")
        roles.remove("Pitching")

    if "Hitting Coach" in roles and "Hitting" in roles:
        roles.remove("Hitting")
    elif "Hitting" in roles:
        roles.append("Hitting Coach")
        roles.remove("Hitting")

    if "Outfield" in roles:
        roles.append("Outfield Coach")
        roles.remove("Outfield")

    if "Infield" in roles and "Infield Coach" not in roles:
        roles.append("Infield Coach")
        roles.remove("Infield")

    # Avoid situations like: "Strength & Conditioning Coach" + "Strength" + "Conditioning"
    if "Strength" in roles and "Strength & Conditioning Coach" not in roles:
        roles.append("Strength & Conditioning Coach")
    if "Strength & Conditioning Coach" in roles:
        for r in ["Strength", "Conditioning"]:
            if r in roles:
                roles.remove(r)
    
    if "Recruiting" in roles:
        roles.remove("Recruiting")

    return roles

def extract_roles(title):
    title = title.lower().strip()

    # Fix merged words
    title = re.sub(r'(?<=[a-z])(?=[A-Z])', ' ', title)

    # Normalize characters
    title = title.replace("&", "/")
    title = title.replace("/", " / ")
    title = re.sub(r"[(),]", " ", title)
    title = re.sub(r"\s+", " ", title)

    parts = [p.strip() for p in title.split("/") if p.strip()]

    roles = []

    for p in parts:
        for key in ROLE_KEYWORDS:
            if key in p:
                canonical = CANONICAL_MAP.get(key, key.title())
                if canonical not in roles:
                    roles.append(canonical)

    # Fallback: if no match found
    if not roles:
        roles.append("Assistant Coach")  # default

    roles = clean_role_list(roles)

    return roles

coaches_df = coaches.copy()
coaches_df['Role List'] = coaches_df['Title'].apply(extract_roles)

In [67]:
coaches_df.to_csv('temp.csv', index=False)